In [69]:
import requests
import googlemaps
import pandas as pd
from geopy.distance import geodesic
import folium
import time
from datetime import datetime



 ### Google Maps API Key

In [52]:
url_loc = "xxxxxxxxxx"
gmaps = googlemaps.Client(key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')


In [3]:
cities = ['Copenhagen', 'Aalborg', 'Arhus']
gyms = ['PureGym Copenhagen', 'PureGym Aalborg', 'PureGym Arhus']
loc_name = cities[0]    

place_details = gmaps.places(loc_name)

### for roads if needed
response = requests.get(url_loc)

In [4]:
print(place_details['results'][0]['geometry']['location'])
print(place_details.keys())

place_results = gmaps.places(query=loc_name)

{'lat': 55.6760968, 'lng': 12.5683372}
dict_keys(['html_attributions', 'results', 'status'])


In [9]:
full_adress = place_details['results']
full_adress

[{'formatted_address': 'Copenhagen, Denmark',
  'geometry': {'location': {'lat': 55.6760968, 'lng': 12.5683372},
   'viewport': {'northeast': {'lat': 55.7270937, 'lng': 12.7342654},
    'southwest': {'lat': 55.615441, 'lng': 12.4533824}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'Copenhagen',
  'photos': [{'height': 1200,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103711582209233100096">markusrobbe</a>'],
    'photo_reference': 'AcJnMuFU04W9VPm9UZx_BUSh1uoPb-n5sKw0M4i-KglwP_ehSF8uq1CybrikmcAp4WKZ8K7l5Wt7DL8qqvGsASw5o5h8zUqjvdkx-uRx8z98siVcuORfUd5ykuYWhRydluQmsvbEMGQZQEIXNbOLcu4E4pCWkLW-S_fKDqmGwE8T4ZFvKbq3',
    'width': 1600}],
  'place_id': 'ChIJIz2AXDxTUkYRuGeU5t1-3QQ',
  'reference': 'ChIJIz2AXDxTUkYRuGeU5t1-3QQ',
  'types': ['locality', 'political']}]

In [48]:
### function to extract time schedule for each day

def day_schedule_periods(weekday_text):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']   
    some_d = {}

    for i in range(len(weekday_text)):
        open = weekday_text[i]['open']
        close = weekday_text[i]['close']  

        open_t = datetime.strptime(open['time'], "%H%M")
        close_t = datetime.strptime(close['time'], "%H%M")

        formatted_time_open  = open_t.strftime("%I:%M%p")   
        formatted_time_close = close_t.strftime("%I:%M%p")  

        day = weekday_text[i]['open']['date']
        day_week = datetime.strptime(day, "%Y-%m-%d").weekday() 

        some_d[days[day_week]] = formatted_time_open + " - " + formatted_time_close
    return some_d

data_list = []

for gym in gyms:   ### for all gyms in the list 
    places_results = gmaps.places(query=gym)

    if places_results['status'] == 'OK' and len(places_results['results']) > 0:
        for place in places_results['results']:
            place_id = place['place_id']
            place_details = gmaps.place(place_id)

            if 'reviews' in place_details['result']:
                location = place_details['result']['geometry']['location']
                lat = location['lat']
                lng = location['lng']

                for review in place_details['result']['reviews']:
                    author_name = review['author_name']
                    rating = review['rating']
                    text = review['text']

                    if 'current_opening_hours' in place_details['result']:
                        weekday_text = place_details['result']['current_opening_hours']['periods']
                        m_f_schedule = day_schedule_periods(weekday_text)
                    else:
                        m_f_schedule = ""
                        s_su_schedule = ""

                    data_list.append({
                        'place_id': place_id,
                        'name': gym,
                        'lat': lat,
                        'lng': lng,
                        'author_name': author_name,
                        'rating': rating,
                        'text': text,
                        'Monday_schedule': m_f_schedule['Monday'],
                        'Tuesday_schedule': m_f_schedule['Tuesday'],
                        'Wednesday_schedule': m_f_schedule['Wednesday'],
                        'Thursday_schedule': m_f_schedule['Thursday'],
                        'Friday_schedule': m_f_schedule['Friday'],
                        'Saturday_schedule': m_f_schedule['Saturday'],
                        'Sunday_schedule': m_f_schedule['Sunday'],
                    })

df = pd.DataFrame(data_list)


In [51]:
#df.to_csv('pure_gym_reviews.csv', index=False)
df

,place_id,name,lat,lng,author_name,rating,text,Monday_schedule,Tuesday_schedule,Wednesday_schedule,Thursday_schedule,Friday_schedule,Saturday_schedule,Sunday_schedule
0,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,Lewis Atkins,2,"Just a very bad gym. Staff don’t really care, ...",05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 11:59PM
1,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,Rune Perstrup,1,An Unhygienic Coronavirus Petri Dish.\n\nI hav...,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 11:59PM
2,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,Mario Piazza,1,In a huge gym there is only one hair dryer and...,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 11:59PM
3,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,Emil Lassen,5,It's a great gym with basically everything you...,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 11:59PM
4,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,Frank Ejby Poulsen,3,"The gym is great, spacious and has everything,...",05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 12:00AM,05:00AM - 11:59PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,ChIJAfU9wBIVTEYRJ7DNdJ0UqFA,PureGym Arhus,56.178129,10.139422,Mohammed Al Ahmadi,2,Very small,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM
206,ChIJAfU9wBIVTEYRJ7DNdJ0UqFA,PureGym Arhus,56.178129,10.139422,Flex Trafik,3,"Nice place to move around a bit, unfortunately...",05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM
207,ChIJAfU9wBIVTEYRJ7DNdJ0UqFA,PureGym Arhus,56.178129,10.139422,MS B,1,Please note that if a subscription is put on h...,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM
208,ChIJAfU9wBIVTEYRJ7DNdJ0UqFA,PureGym Arhus,56.178129,10.139422,Orla Olsen,5,Good machines and great music. You really want...,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM,05:00AM - 10:00PM


In [80]:
### function to calculate the distance between 2 points
### remember that there are 5 reviews for each gym, so if you want to calculate the distance between 2 gyms, you need to bare this in mind

def distance_calc(lat1, lon1, lat2, lon2):

    distance_calculator = geodesic((lat1, lon1), (lat2, lon2))

# Calculate the distance in meters
    distance_in_meters = distance_calculator.meters 

    return distance_in_meters

print(distance_calc(df.iloc[0]['lat'], df.iloc[0]['lng'] , df.iloc[6]['lat'], df.iloc[6]['lng'] ))  ### every 5 gyms are the same, but the author, review and rating are different



2035.1611613715165


### Supermarkets/parks


In [64]:

# Define the type of place you want to go after/before gym
attractions = ['supermarket', 'park']
place_type = "supermarket"

# Define the location (latitude and longitude) and radius of the search area
location = (55.669812, 12.547390)
radius = 1500  # Specify the radius in meters

places_results = gmaps.places_nearby(location=location, radius=radius, type=place_type)

for atr in attractions:
    for place in places_results['results']:
    # Extract information about each supermarket or park
        place_name = place['name']
        place_address = place.get('vicinity', '')
        print(place_name)



Føtex
REMA 1000
SuperBrugsen
SuperBrugsen
Irma Supermarket
REMA 1000
Min Købmand
Lidl
Netto
føtex food
Aldi
Ah Express
Netto
Netto
Netto
Netto
føtex Carlsberg Byen
China Town Market
Netto
Lidl
Føtex
REMA 1000
SuperBrugsen
SuperBrugsen
Irma Supermarket
REMA 1000
Min Købmand
Lidl
Netto
føtex food
Aldi
Ah Express
Netto
Netto
Netto
Netto
føtex Carlsberg Byen
China Town Market
Netto
Lidl
